<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2025-02-27 13:46:40--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.185.43.25, 188.185.48.194, 188.185.45.92, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.25|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1199620/files/SO_vectors_200.bin [following]
--2025-02-27 13:46:40--  https://zenodo.org/records/1199620/files/SO_vectors_200.bin
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  16.4MB/s    in 77s     

2025-02-27 13:47:56 (18.1 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [2]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [3]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [4]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### ***Вопрос 1:***
* Входит ли слово `cat` в топ-5 близких слов к слову `dog`? Какое место оно занимает?


In [5]:
# method most_simmilar
wv_embeddings.most_similar('dog')

[('animal', 0.8564180135726929),
 ('dogs', 0.7880866527557373),
 ('mammal', 0.7623804211616516),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665),
 ('feline', 0.7392398715019226),
 ('bird', 0.7315488457679749),
 ('animal1', 0.7219215631484985),
 ('doggy', 0.7213349342346191),
 ('labrador', 0.7209131717681885)]

In [6]:
word = 'cat'
count = 0
other_words = []
for i,w in enumerate(wv_embeddings.most_similar('dog')):
  if w[0] == word and i < 5:
    print(f'Слово {word} входит в топ-5 близких слов к слову dog')
    break
  else:
    count += 1
    if count == 5:
      print(f'Слово {word} НЕ входит в топ-5 близких слов к слову dog')
      break

Слово cat НЕ входит в топ-5 близких слов к слову dog


### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [7]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()

In [8]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """
    count = 0
    q_emb_sum = np.zeros(shape=dim)

    for w in tokenizer.tokenize(question.lower()):
      if w in embeddings:
        q_emb_sum += embeddings[w]
        count += 1
    return q_emb_sum / count if count > 0 else q_emb_sum

Теперь у нас есть метод для создания векторного представления любого предложения.

#### ***Вопрос 2:***

* Какая третья (с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [9]:
sentence = 'I love neural networks'

In [10]:
round(question_to_vec(question=sentence, embeddings=wv_embeddings, tokenizer=tokenizer)[2], 2)

-1.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

DCG@K  измеряет качество ранжирования, учитывая не только факт наличия правильного ответа в топ-K, но и ***его точную позицию***.

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### ***Вопрос 3***:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)


In [11]:
ranq = 9
K = 10
DCG10score = (1 / np.log2(1 + ranq)) * (ranq <= K)
print(f'DCG@10 для rank_q′i=9 равно {round(DCG10score,1)}')

DCG@10 для rank_q′i=9 равно 0.3


#### Более сложный пример оценок

Рассмотрим пример с $ N > 1 $, где $ N = 3 $ (три вопроса) и для каждого вопроса заданы позиции их дубликатов. Вычислим метрики **Hits@K** для разных значений $ K $.

---

- $ N = 3 $: Три вопроса ($ q_1, q_2, q_3 $).
- Для каждого вопроса известна позиция его дубликата ($ \text{rank}_{q'_i} $):
  - $ \text{rank}_{q'_1} = 2 $,
  - $ \text{rank}_{q'_2} = 5 $,
  - $ \text{rank}_{q'_3} = 1 $.

Мы будем вычислять **Hits@K** для $ K = 1, 5 $.

---

**Для $ K = 1 $:**

Подставим значения:
$$
\text{Hits@1} = \frac{1}{3} \cdot \left( [\text{rank}_{q'_1} \leq 1] + [\text{rank}_{q'_2} \leq 1] + [\text{rank}_{q'_3} \leq 1] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 1 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 1 $ → $ 1 $.

Сумма:
$$
\text{Hits@1} = \frac{1}{3} \cdot (0 + 0 + 1) = \frac{1}{3}.
$$

$$
\boxed{\text{Hits@1} = \frac{1}{3}}.
$$

---

**Для $ K = 5 $:**

Подставим значения:
$$
\text{Hits@5} = \frac{1}{3} \cdot \left( [\text{rank}_{q'_1} \leq 5] + [\text{rank}_{q'_2} \leq 5] + [\text{rank}_{q'_3} \leq 5] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 5 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 5 $ → $ 1 $.

Сумма:
$$
\text{Hits@5} = \frac{1}{3} \cdot (1 + 1 + 1) = 1.
$$

$$
\boxed{\text{Hits@5} = 1}.
$$

#### ***Вопрос 4:***
* Найдите максимум `Hits@47 - DCG@1`?

**Ответ: 1**

1. Hits@47 = 1 для N=1 во всех случаях, когда rank_q1' дубликата <= 47.
В нашем случае rank_q1' = 2.
2. DCG@1 = 0 для N=1: 1 / log2(1 + rank_q1') * [rank_q1' <= 1] =
= 1 / log2(3) * [2 <= 1] = 0.
3. Таким образом, во всех случаях, когда rank_q1' принадлежит диапазону (1;47]: максимум (Hits@47 - DCG@1) = 1. При этом в самом "близком" случае rank_q1' = 1 максимум (Hits@47 - DCG@1) = max(1-1) = 0, в этом случае значение двух метрик совпадает.

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$.$dup\_ranks$ является списком, который содержит рейтинги дубликатов (их позиции в ранжированном списке).

К примеру для <font color='red'>"Что такое язык python?"</font> $dup\_ranks = [2]$.

In [12]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    hits_value = np.mean(np.array(dup_ranks) <= k)
    return hits_value

In [13]:
dup_ranks = [2]

k = 1
hits_value = hits_count(dup_ranks, k)
print(f"Hits@1 = {hits_value}")

k = 4
hits_value = hits_count(dup_ranks, k)
print(f"Hits@4 = {hits_value}")

Hits@1 = 0.0
Hits@4 = 1.0


In [14]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    ranks = np.array(dup_ranks)
    flags =  (ranks <= k)
    dcg_value = np.mean(1 / np.log2(1 + ranks) * flags)
    return dcg_value

In [15]:
# Пример списка позиций дубликатов
dup_ranks = [2]

# Вычисляем DCG@1
dcg_value = dcg_score(dup_ranks, k=1)
print(f"DCG@1 = {dcg_value:.3f}")

# Вычисляем DCG@4
dcg_value = dcg_score(dup_ranks, k=4)
print(f"DCG@10 = {dcg_value:.3f}")

DCG@1 = 0.000
DCG@10 = 0.631


Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [18]:
import pandas as pd

In [16]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [candidates_ranking[answer].index(copy_answers[answer]) + 1 for answer in range(len(copy_answers))]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])


Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [19]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [20]:
!unzip /content/stackoverflow_similar_questions.zip

unzip:  cannot find or open /content/stackoverflow_similar_questions.zip, /content/stackoverflow_similar_questions.zip.zip or /content/stackoverflow_similar_questions.zip.ZIP.


Считайте данные.

In [21]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
      data.append(line.strip().split('\t'))
    return data

Нам понадобиться только файл validation.

In [36]:
validation_data = read_corpus('/content/validation.tsv')

Кол-во строк

In [37]:
len(validation_data)

3760

Размер нескольких первых строк

In [29]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [30]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [31]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    q_vec = [question_to_vec(question, embeddings, tokenizer, dim)]
    candidates_vec = [question_to_vec(sent, embeddings, tokenizer, dim) \
                      for sent in candidates]
    cosine_sim = np.argsort(cosine_similarity(q_vec, candidates_vec).ravel())[::-1]
    return [(i, candidates[i]) for i in cosine_sim]

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [32]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [33]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks,end='')
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]
[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]


Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [34]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(0, 'Getting all list items of an unordered list in PHP'), #скрыт
            (2, 'select2 not displaying search results'), #скрыт
            (1, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### ***Вопрос 5:***
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

### Ответ
Последовательность начальных индексов для эксперимента 2 - `021`

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [35]:
from tqdm.notebook import tqdm

In [38]:
wv_ranking = []
max_validation_examples = 1000
with tqdm(desc="Corpus processed", total=max_validation_examples) as pbar_outer:
  for i, line in enumerate(validation_data):
      if i == max_validation_examples:
          break
      q, *ex = line
      ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
      wv_ranking.append([r[0] for r in ranks].index(0) + 1)
      pbar_outer.update(1)

Corpus processed:   0%|          | 0/1000 [00:00<?, ?it/s]

In [39]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.415 | Hits@   1: 0.415
DCG@   5: 0.502 | Hits@   5: 0.582
DCG@  10: 0.524 | Hits@  10: 0.650
DCG@ 100: 0.570 | Hits@ 100: 0.874
DCG@ 500: 0.583 | Hits@ 500: 0.973
DCG@1000: 0.586 | Hits@1000: 1.000


Из формул выше можно понять, что

- $ \text{Hits@K} $ **монотонно неубывающая функция** $ K $, которая стремится к 1 при $ K \to \infty $.

- $ \text{DCG@K} $ **монотонно неубывающая функция** $ K $, но рост замедляется с увеличением $ K $ из-за убывания веса $ \frac{1}{\log_2(1 + \text{rank}_{q'_i})} $.

### Эмбеддинги, обученные на корпусе похожих вопросов

In [45]:
train_data = read_corpus('/content/train.tsv')

In [43]:
train_data[:2]

[['converting string to list',
  'Convert Google results object (pure js) to Python object'],
 ['Which HTML 5 Canvas Javascript to use for making an interactive drawing tool?',
  'Event handling for geometries in Three.js?']]

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

***Рассмотрим подробнее*** данное склеивание.

1. Каждая строка из train_data разбивается на вопрос (question) и список кандидатов.

2. Для каждого кандидата вопрос склеивается с ним в одну строку.

3. Склеенная строка (combined_text) токенизируется, и полученный список токенов добавляется в общий корпус (corpus).

***Пример***

    Вопрос: "What is Python?"
    Кандидаты: ["Python is a programming language", "Java is another language"]
    Склеенные строки:
        "What is Python? Python is a programming language"
        "What is Python? Java is another language"
         
    Токенизированные списки:
        ['what', 'is', 'python', 'python', 'is', 'a', 'programming', 'language']
        ['what', 'is', 'python', 'java', 'is', 'another', 'language']
         
     

In [47]:
train_data[111258]

['Determine if the device is a smartphone or tablet?',
 'Change imageView params in all cards together']

In [46]:
new_corpus = [couple[0] + str(' ') + couple[1] for couple in train_data]
new_corpus[:2]

['converting string to list Convert Google results object (pure js) to Python object',
 'Which HTML 5 Canvas Javascript to use for making an interactive drawing tool? Event handling for geometries in Three.js?']

Посчитаем минимальную и средние длины предложений в получившемся корпусе

In [48]:
corpus_lens = []
for i in tqdm(range(len(new_corpus))):
  corpus_lens.append(len(new_corpus[i].split()))

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [49]:
corpus_lens[:10]

[13, 19, 15, 14, 20, 16, 14, 35, 11, 25]

In [50]:
mean_len = np.mean(corpus_lens)
min_len = min(corpus_lens)
print(f'Средняя длина вопроса - {round(mean_len, 2)} токенов. ')
print(f'Минимальная длина вопроса в корпусе - {min_len} токена. ')

Средняя длина вопроса - 16.99 токенов. 
Минимальная длина вопроса в корпусе - 2 токена. 


Видим, что после "склейки" средняя длина вопросов стала значительной, при этом до "склейки" в корпусе содержались предложения, состоящие лишь из 1го токена.

In [51]:
proc_words = [tokenizer.tokenize(text) for text in tqdm(new_corpus)]

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [52]:
proc_words[0]

['converting',
 'string',
 'to',
 'list',
 'Convert',
 'Google',
 'results',
 'object',
 'pure',
 'js',
 'to',
 'Python',
 'object']

Исходя из среднего и мин размера вопроса в корпусе, укажем размер окна = 7, при этом размер скрытого слоя укажем равным 300

In [53]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(proc_words, # data for model to train on
                 vector_size=300,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=7).wv

In [54]:
print(embeddings_trained.index_to_key[:5])


['to', 'in', 'a', 'How', 'the']


In [55]:
wv_ranking = []
max_validation_examples = 1000
dim = 300
with tqdm(desc="Corpus processed", total=max_validation_examples) as pbar_outer:
  for i, line in enumerate(validation_data):
      if i == max_validation_examples:
          break
      q, *ex = line
      ranks = rank_candidates(q, ex, embeddings_trained, tokenizer, dim=300)
      wv_ranking.append([r[0] for r in ranks].index(0) + 1)
      pbar_outer.update(1)

Corpus processed:   0%|          | 0/1000 [00:00<?, ?it/s]

In [56]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.262 | Hits@   1: 0.262
DCG@   5: 0.339 | Hits@   5: 0.405
DCG@  10: 0.360 | Hits@  10: 0.471
DCG@ 100: 0.413 | Hits@ 100: 0.732
DCG@ 500: 0.437 | Hits@ 500: 0.918
DCG@1000: 0.446 | Hits@1000: 1.000


Видим, что качество модели, обученной на полученных эмбеддингах, довольно существенно уступает результатам ранжирования с использованием ранее обученного словаря wv_embeddings.

**Попробуем более сложную кастомную токензацию с использвоанием библиотеки NLTK**:

1) приведем все слова в корпусе к нижнему регистру;

2) удалим стоп-слова;

3) проведем лемматизацию

При этом знаки пунктуации и проч служебные символы удалять не будем.

In [61]:
import nltk
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
#nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab')
stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [58]:
class MyComplexTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
      text = text.lower()
      lemmatizer = WordNetLemmatizer()
      tokenized = nltk.word_tokenize(text)
      filtered_sentence = [w for w in tokenized if not w in stopwords]
      lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in filtered_sentence])
      text_only = re.sub(r"[^a-z0-9!@#\$%\^\&\*_\-,\.' ]", ' ', lemmatized_output)
      final = ' '.join(text_only.split())
      return final

In [59]:
tokenizer = MyComplexTokenizer()

In [62]:
wv_ranking = []
max_validation_examples = 1000
dim = 300
with tqdm(desc="Corpus processed", total=max_validation_examples) as pbar_outer:
  for i, line in enumerate(validation_data):
      if i == max_validation_examples:
          break
      q, *ex = line
      ranks = rank_candidates(q, ex, embeddings_trained, tokenizer, dim=300)
      wv_ranking.append([r[0] for r in ranks].index(0) + 1)
      pbar_outer.update(1)

Corpus processed:   0%|          | 0/1000 [00:00<?, ?it/s]

In [63]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.110 | Hits@   1: 0.110
DCG@   5: 0.135 | Hits@   5: 0.161
DCG@  10: 0.150 | Hits@  10: 0.207
DCG@ 100: 0.198 | Hits@ 100: 0.451
DCG@ 500: 0.242 | Hits@ 500: 0.801
DCG@1000: 0.263 | Hits@1000: 1.000


Видим, что после указанной нормализации словаря качество метрик значительно ухудшилось. Предположим, что нормализация слов уменьшает количество значимой информации, необходимой для обучения эмбеддингов: видимо, важен и регистр и разные словоформы. В таких условиях логично:

1) увеличить размер окна;

2) при этом размер скрытого слоя снизить, дабы модель смогла усвоить как можно более значимые признаки;

3) с помощью модели Skip-Gram постараемся получить значимую инфу о контексте каждого токена (архитектура Skip-Gram лучше CBOW позволяет представлять редкие слова или фразы).

In [64]:
embeddings_trained = Word2Vec(proc_words, # data for model to train on
                 vector_size=200,                 # embedding vector size
                 min_count=9,             # consider words that occured at least 5 times
                 window=10,
                        sg=1).wv #

In [65]:
tokenizer = MyTokenizer()

In [66]:
wv_ranking = []
max_validation_examples = 1000
dim = 200
with tqdm(desc="Corpus processed", total=max_validation_examples) as pbar_outer:
  for i, line in enumerate(validation_data):
      if i == max_validation_examples:
          break
      q, *ex = line
      ranks = rank_candidates(q, ex, embeddings_trained, tokenizer, dim=dim)
      wv_ranking.append([r[0] for r in ranks].index(0) + 1)
      pbar_outer.update(1)

Corpus processed:   0%|          | 0/1000 [00:00<?, ?it/s]

In [67]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.430 | Hits@   1: 0.430
DCG@   5: 0.510 | Hits@   5: 0.578
DCG@  10: 0.528 | Hits@  10: 0.635
DCG@ 100: 0.571 | Hits@ 100: 0.844
DCG@ 500: 0.585 | Hits@ 500: 0.951
DCG@1000: 0.590 | Hits@1000: 1.000


**Видим, что наши предположения оказались верны: получившиеся эмбеддинги даже чуть превосходят результаты ранжирования с использованием ранее обученного словаря wv_embeddings.**

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


1. Опытным путем выяснили, что лучшее качество для данных текстов дает довольной минимилистичный, кастомный способ токенизации, когда остаются только буквенные токены с сохранением их словоформ. Видимо, сказывается специфика текстов StackOverFlow: наполненность техническими словами и смыслами при маленькой длине предложений (до "склейки" < 10, тогда как из открытых источников ср длина предложения в русском языке ~ 20 токенов), широкое использование аббревиатур и обозначений, учитывающих регистр, и тп.

2. Исходя из сказанного выше, понятно, что нормализация слов будет только ухудшать качество.

3. После "склейки" пар слов в корпусе и исследования среднией и минимальной длин вопросов в получившемся корпусе был подобран диапазон для поиска оптимальных гиперпарамтеров, главным образом, размер окна.
В результате перебора различных вариантов подобраны следующие гиперпараметры:

  *   `vector_size`=200,
  *   `min_count`=9,             
  *   `window`=10,
  *   архитектура - Skip-Gram
Обоснование, почему такие эмбеддинги могут работать лучше, приведено выше.

4. Качество решения, действительно, получилось не очень. При этом есть подозрения, что, как бы мы не старались, в рамках архитектуры word2vec мы не добьемся существенного увеличения качества. По-прежнему остаются недостатки:

  *   Обучение на уровне слов: нет информации о предложении или контексте, в котором используется слово.
  *   Совместная встречаемость игнорируется. Модель не учитывает то, что слово может иметь различное значение в зависимости от контекста использования.
  *   Не очень хорошо обрабатывает неизвестные и редкие слова.

5. Наиболее очевидное решение использовать те модели, которые минимизируют указанные недостатки: так модель GloVe (Global Vectors) на базе того же word2vec учитывает частоту встречаемости слов и, как правило, опережает word2vec на большинстве бенчмарков. Недостаток с неизвестными и редкими словами может "закрыть" fastText, которая к основной модели word2vec добавлет модель символьных n-грамм (способна генерировать эмбеддинги и для неизвестных слов).